In [78]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import mysql.connector
from sklearn.metrics import accuracy_score


with open('pss.txt') as file:
    lines = file.readlines()
    lines = [line.rstrip() for line in lines]
password = lines[0]
db = lines[1]

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password=password,
    database=db,
    use_pure=True
)

my_cursor = mydb.cursor()
steps_title_list = []
d_steps_title = {}
university_name_list = []
d_uni_name = {}

def convert_nans(df):
    df['steps_title'] = df['steps_title'].fillna(6)
    df['age'] = df['age'].fillna(-1)
    df['skill_exists'] = df['skill_exists'].fillna(-1)
    df['marriage_status'] = df['marriage_status'].fillna(-1)
    df['lanquage_exists'] = df['lanquage_exists'].fillna(-1)
    df['degree'] = df['degree'].fillna(-1)
    df['university'] = df['university'].fillna(-1)
    df['gpa'] = df['gpa'].fillna(-1)
    df['job_contract_type'] = df['job_contract_type'].fillna(-1)
    df['edu_interval'] = df['edu_interval'].fillna(-1)
    df['skill_require'] = df['skill_require'].fillna(-1)
    df['exprience_exists'] = df['exprience_exists'].fillna(-1)
    df['skill_exists'] = df['skill_exists'].fillna(-1)
    df['gender'] = df['gender'].fillna(-1)




my_cursor.execute('select distinct steps_title from useful_data')
for item in my_cursor:
    steps_title_list.append(item)

my_cursor.execute('select distinct university from useful_data where steps_title = "استخدام شده" or steps_title = "رد شده"')
for item in my_cursor:
    university_name_list.append(item)


for title in steps_title_list:
    if title[0] == 'نیازمند تعیین وضعیت':
        d_steps_title[title[0]] = 0
    elif title[0] == 'تایید برای مصاحبه':
        d_steps_title[title[0]] = 1
    elif title[0] == 'استخدام شده':
        d_steps_title[title[0]] = 2
    elif title[0] == 'رد شده':
        d_steps_title[title[0]] = 3
    elif title[0] == 'انصراف از مصاحبه':
        d_steps_title[title[0]] = 4
    else:
        d_steps_title[title[0]] = 6

i = 0
for title in university_name_list:
    d_uni_name[title] = i
    i += 1

np.random.seed(0)
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_train['steps_title'] = df_train['steps_title'].map(d_steps_title)
df_train['university'] = df_train['university'].map(d_uni_name)
df_test['university'] = df_test['university'].map(d_uni_name)
df_test['steps_title'] = df_test['steps_title'].map(d_steps_title)
convert_nans(df_train)
convert_nans(df_test)
df_train.head()



# df_train['is_train'] = np.where((df_train['steps_title'] == 2) | (df_train['steps_title'] == 3), True, False)
# train , test = df_train[df_train['is_train'] == True] , df_train[df_train['is_train'] == False]
print('number of training data: ' , len(df_train))
print('number of test data: ' , len(df_test))
features_train = df_train.columns[:13]
features_test = df_test.columns[:13]

clf = RandomForestClassifier(n_jobs=2 , random_state=0)
clf.fit(df_train[features_train] , df_train['steps_title'])
y = clf.predict(df_test[features_test]).tolist()
x = df_test['steps_title'].tolist()
print(y)
print(x)
accuracy_score(x, y)
# pd.crosstab(x, y , rownames='actual step', colnames='predict step')

number of training data:  17912
number of test data:  108
[3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 3, 3, 2, 3, 2, 2, 3, 3, 3, 2, 3, 3, 3, 2, 3, 2, 3, 3, 2, 2, 3, 3, 3, 3, 3, 3, 3, 2, 3, 2, 3, 2, 2, 3, 2, 3, 3, 2, 2, 3, 3, 2, 3, 3, 2, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 3, 3, 2, 3, 2, 2, 2, 3, 2, 3, 3, 3, 3, 2, 3, 3, 3, 3, 2, 3, 3, 2, 2, 3, 3, 3]
[3, 3, 3, 3, 2, 3, 2, 3, 3, 3, 3, 2, 3, 3, 2, 2, 3, 2, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 3, 2, 2, 3, 3, 2, 2, 3, 2, 2, 3, 2, 2, 2, 3, 3, 3, 3, 2, 3, 2, 3, 2, 3, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 3, 2, 2, 3, 3, 2, 3, 3, 2, 3, 3, 3, 3, 3, 2, 3, 3, 2, 2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 3, 3, 3, 2, 3, 3, 2, 2, 3, 2, 3]


0.7962962962962963